In [ ]:
import ast
import base64
import pickle

import requests
from pydantic import BaseModel, HttpUrl


class InputData(BaseModel):
    text: str = "Default input text"
    parameter1: int = 0
    parameter2: bool = False


class PhotonClient:
    DEFAULT_URL = "http://localhost/photonapi"

    def __init__(self, api_key: str, url: HttpUrl | None = None):
        self.api_key = api_key
        self.url = url if url is not None else HttpUrl(self.DEFAULT_URL)
        self.headers = {"key": self.api_key, "Content-Type": "application/json"}

    @staticmethod
    def parse_sse(response):
        """Parse Server-Sent Events from a response."""
        for line in response.iter_lines():
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    yield line[6:]  # Remove 'data: ' prefix

    def run(self, input_data: InputData | None = None):
        if input_data is None:
            input_data = InputData()
        try:
            # Send the initial POST request with JSON data
            response = requests.post(
                str(self.url), json=input_data, headers=self.headers, stream=True
            )
            response.raise_for_status()  # Raise an exception for bad status codes
            for data in self.parse_sse(response):
                try:
                    event = ast.literal_eval(data)
                    # event must be a dictionary: parsing it into a dictionary
                    if "pickled_response" in event.keys():
                        binary_data = base64.b64decode(event["pickled_response"])
                        response = pickle.loads(binary_data)
                        # print(f"Received pickled response with keys: {response.keys()}")
                        print("Received pickled response!")
                        return response
                    else:
                        print(f"Error1: Received data: {data}")
                except:
                    print(f"Error2: Received non-dict data: {data}")
        except requests.exceptions.RequestException as e:
            print(f"An error occurred while making the request: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [ ]:
# photonclient = PhotonClient(url="http://localhost/photonapi", api_key = "iTL4i1LiqqxoY7W5f4hmoNZIFrZAjSn")


from IPython.display import Image as IPImage
from IPython.display import display

photonclient = PhotonClient(
    url="http://209.38.182.153/photonapi", api_key="iTL4i1LiqqxoY7W5f4hmoNZIFrZAjSn"
)

prompt = {"prompt": "a 2x2 mmi"}
r = photonclient.run(input_data=prompt)


print(r.keys())
print(r["netlist"])
IPImage(r["graph_png"])

In [ ]:
from IPython.display import Image as IPImage

print(r.keys())

IPImage(r["graph_png"])
print(r["netlist"])

In [ ]:
type(r)

In [ ]:
from IPython.display import Image as IPImage


def pretty_display(response_dict):
    for key, item in response_dict.items():
        if "png" in key:
            display(IPImage(response_dict[key]))
        if "png" in key:
            display(IPImage(response_dict[key]))
        print(key, type(item))


pretty_display(r)